# <center> 基于编解码框架的服饰图像描述生成中期报告

## 一、详细设计

对于此服饰图像描述生成实验，我们定义了以下几个模块来实现不同的功能：
- 1. 数据预处理
- 2. 视觉Transformer + Transformer解码器模型
- 3. 网格/区域表示、Transformer编码器+Transformer解码器
- 4. 评估指标

### 1. 数据预处理

#### 1.1 下载数据集
- 我们使用的数据集为flickr8k([下载地址](https://www.kaggle.com/adityajn105/flickr8k))。
- 下载解压后，将其图片放在指定目录(本代码中将该目录设置为../data/flickr8k)下的images文件夹里。
- 数据集包括8000张图片，每张图片对应5个句子描述。
- 数据集划分采用Karpathy提供的方法([下载地址](http://cs.stanford.edu/people/karpathy/deepimagesent/caption_datasets.zip))，下载解压后，将其中的dataset_flickr8k.json文件拷贝到指定目录下。该划分方法将数据集分成3个子集：6,000张图片和其对应句子描述组成训练集，1,000张图片和描述为验证集，剩余的1,000张图片和描述为测试集。

#### 1.2 整理数据集
数据集下载完成后，我们需要对其进行处理，以适合之后构造的PyTorch数据集类读取。
- 对于文本描述，我们首先构建词典，然后根据词典将文本描述转化为向量。
- 对于图像，我们这里仅记录文件路径。
- 如果机器的内存和硬盘空间就比较大，这里也可以将图片读取并处理成三维数组，这样在模型训练和测试的阶段，就不需要再直接读取图片。

#### 1.3 定义数据集类
在准备好的数据集的基础上，我们需要进一步定义PyTorch Dataset类，以使用PyTorch DataLoader类按批次产生数据。PyTorch中仅预先定义了图像、文本和语音的单模态任务中常见的数据集类。因此，我们需要定义自己的数据集类。
在PyTorch中定义数据集类非常简单，仅需要继承torch.utils.data.Dataset类，并实现\_\_getitem\_\_和\_\_len\_\_两个函数即可。

#### 1.4 批量读取数据
利用刚才构造的数据集类，借助DataLoader类构建能够按批次产生训练、验证和测试数据的对象。


### 2. 视觉Transformer + Transformer解码器模型

#### 2.1 模型架构：
此模型结构包括一个视觉Transformer作为编码器，用于处理图像数据，以及一个Transformer解码器，用于将编码后的图像数据转换为文本描述。

#### 2.2 模块划分：

- **视觉Transformer编码器**：
     - **自注意力机制**：编码器使用自注意力机制来处理图像。它首先将图像划分为多个小块，然后对这些块进行线性变换，生成一组“tokens”。
     - **多头注意力**：在自注意力机制中，使用多头注意力来并行捕捉图像不同区域的信息。每个注意力头学习图像的不同方面，如纹理、颜色或形状。
     - **位置编码**：由于自注意力机制本身不具备捕捉顺序的能力，因此引入位置编码来保留图像块的空间位置信息。

- **Transformer解码器**：
     - **初始化**：解码器以特殊的开始标记（如"start"）初始化，随后逐步生成文本描述。
     - **多头自注意力**：解码器的每一层都包括一个多头自注意力机制，用于聚焦先前生成的词元。
     - **编码器-解码器注意力**：解码器的每一层还包含一个编码器-解码器注意力机制，用于将编码器的输出（图像的特征表示）与当前生成的文本相关联。
     - **输出生成**：解码器的最后一层使用线性层和softmax函数生成下一个词的概率分布。根据这个分布选取概率最高的词作为输出。


#### 2.3 接口设计：
视觉Transformer的输出被直接传递给Transformer解码器，以便生成文本描述。  

#### 2.4 技术方案：  
- 使用视觉Transformer来捕捉图像的关键视觉信息。
- Transformer解码器使用编码器的输出来生成描述性文本。
- 在定义编码器和解码器完成之后，我们就很容易构建图像描述模型了。仅需要在初始化函数时声明编码器和解码器，然后在前馈函数实现里，将编码器的输出和文本描述作为解码器的输入即可。


### 3. 网格/区域表示、Transformer编码器+Transformer解码器

#### 3.1 模型架构：
此模型利用网格或区域表示方法来处理图像，然后通过Transformer编码器和解码器生成描述。

#### 3.2 模块划分：

- **网格/区域表示处理**：
    - 使用先进的卷积神经网络（如ResNet）提取图像的特征。
    - 将特征图划分为不同的网格或区域，每个网格或区域代表图像的一部分。

- **Transformer编码器**：
    - 对于每个网格或区域的特征向量，编码器使用自注意力机制进行处理。
    - 编码器在处理时，同时考虑图像的全局内容和局部细节。

- **Transformer解码器（具体实现参考示例）**：
    - 解码器使用结合注意力的GRU单元。
    - 加性注意力机制：具体形式为$ W_2^T \tanh(W_1 [q_i; k_j])$，其中$ W_1$ 和 $W_2$ 是学习参数。
    - 首先，$W_1$ 将查询 $q$ 和键 $k$ 映射到隐藏层表示空间。
    - 然后，映射后的查询和键相加，通过非线性变换后，使用 $W_2$ 将其映射为一组数值。
    - 使用softmax函数获取归一化的相关性分数。
    - 最终，以这些分数为权重，对值进行加权求和，生成文本描述的每个词。


#### 3.3 接口设计：


#### 3.4 技术方案： 
- 利用网格或区域表示来详细捕捉图像的局部特征。
- 通过基于Transformer架构的编码器和解码器处理这些特征，根据图像不同部分的特征生成文本描述
- 基于Pytorch框架加载数据集、计算梯度、训练模型


### 4.评估指标

#### 4.1 CIDEr
- **用途**：CIDEr主要用于图像描述生成任务的评估。
- **原理**：CIDEr通过考虑多个参考描述之间的一致性来评估生成描述的质量。它使用TF-IDF权重和相似性分数来衡量文本之间的相似性。
- **优点**：CIDEr在图像描述生成任务中具有很好的性能，能够捕获多样性和一致性。
- **限制**：CIDEr可能在其他任务上表现不如其他指标。

#### 4.2 SPICE
- **用途**：SPICE主要用于图像描述生成任务，强调对生成描述的语义内容进行准确评估。
- **原理**：SPICE使用语义分析和语义提取，以评估生成文本中的概念和关系是否准确反映了图像内容。
- **优点**：SPICE注重语义准确性，适用于需要更深层次语义理解的任务。
- **限制**：计算复杂度较高，需要大量语义分析。

## 二、已完成工作

**1. 准备工作**：
阅读示范代码并进行调试，在A3000上初步训练得到BLUE-4评分0.25。每个成员在这一过程中熟悉了针对该问题的数据处理、网络构建的方法，大致了解训练过程中可能出现的问题及处理方法，比如张量不匹配。

**2. 数据预处理**：
下载DeepFashion-MultiModal数据集，仿照示范代码中create_dataset类的方式对数据集进行划分，构造词典并分别整理文字和图像数据，便于模型输入。我们考虑将图片分批装载进内存，而不是使用时全部重新装载，以提高模型训练速度。目前数据集已能够可视化展示图片及相应的文字描述。

**3. 模型实现**：
实现了标准的基于Transformer架构的编码器和解码器，并准备尝试不同的超参数设置，对该任务进行优化。目前基于Transformer架构的编码器和解码器已成功适配主体训练流程。

## 三、初步结论

1. 相比与基于GRU的方法，基于Transformer的方法学习速度更快，经过少量训练即可在BLUE-4上达到较高评分。特别地，对于稍微复杂的场景，基于Transformer的方法能够较好地捕捉图片内的复杂的语义关联，生成多样化的文本描述。
2. 相比于将图片整体输入模型，将图片划分为网格区域的方法更有助于模型关注图片的不同部分，从而为图片中物体的不同部分生成相对应的文字描述。网格区域不宜划分地过小，这样计算量会过大，但也不宜划分地过大，这样会导致模型难以捕捉细微的语义。
3. 解码的过程中，需要把图片的整体语义表示传输给解码器的每一层，这样才能让解码器考虑到图片的整体语义特征，从而更好地解读局部信息。

## 四、问题及可能的解决方案

**1. 问题：**如何设置模型的结构和超参数？

**1.1 解决方案：**首先尝试为图像划分合适大小的局部区域，然后尝试基于图像的区域数目和文本描述的数量，确定模型的大致尺寸，然后通过实验尝试找出最有参数。

**2. 问题：**

## 五、后续工作计划

**1. 模型调优：**
继续优化模型参数，提高图像描述的准确性和流畅性。

**2. 更广泛的数据集测试：**
在更多数据集上测试模型，以验证其泛化能力。

**3. 性能评估：**
深入分析和比较两种模型在不同指标上的表现。

**4. 技术创新：**
若时间和技术允许，完成进一步的技术创新工作：
- 默认实现的交叉熵损失和评测指标不一致，请实现基于强化学习的损失函数，直接优化评测指标。
- 微调多模态预训练模型或多模态大语言模型，并测试性能。 
- 利用训练的服饰图像描述模型和多模态大语言模型，为真实背景的服饰图像数据集增加服饰描述和背景描述，构建全新的服饰图像描述数据集并在新数据集上重新训练服饰图像描述模型。

## 六、总结

目前项目进展顺利，我们已经完成了模型的实现和初步评估。后续工作将集中在进一步优化模型性能、细化错误分析，并根据需要进行模型结构和参数调整，以提高模型在生成图像描述方面的准确性和相关性。